In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from manipulate_data import *
from recurrent import *
from ARMA import *
from RF import *

def train(frequency_index, frequencies, frequencies_number_of_samples):

    print(f'Frequency: {frequencies[frequency_index]}')
    number_of_study_periods, study_periods, Data, dates = creating_study_periods(frequencies,\
                                                                                 frequencies_number_of_samples,\
                                                                                 frequency_index)
    
    RF_parameters, RF_mse, RF_predictions = train_RF(number_of_study_periods, study_periods, frequency_index, frequencies,\
                                                 frequencies_number_of_samples)

In [2]:
model_names = ['ARMA', 'LSTM', 'GRU']

frequencies = ['D', '60min', '15min', '5min', 'min']
frequencies_number_of_samples = np.multiply([1, 7, 26, 78, 390], 252)

In [3]:
for frequency_index in range(5):
    train(frequency_index, frequencies, frequencies_number_of_samples)

Frequency: D
Period: 0
|   iter    |  target   | look_back |
-------------------------------------
|  1        | -0.000318 |  22.8     |
|  2        | -0.000330 |  11.2     |
|  3        | -0.000315 |  40.0     |
Period time: 0.23 minutes
Period: 1
|   iter    |  target   | look_back |
-------------------------------------
|  1        | -9.551e-0 |  38.08    |
|  2        | -0.000109 |  14.41    |
|  3        | -0.00042  |  1.002    |
Period time: 0.21 minutes
Period: 2
|   iter    |  target   | look_back |
-------------------------------------
|  1        | -0.000122 |  39.84    |
|  2        | -0.000133 |  4.421    |
|  3        | -0.000203 |  1.0      |
Period time: 0.26 minutes
Period: 3
|   iter    |  target   | look_back |
-------------------------------------
|  1        | -8.008e-0 |  21.34    |
|  2        | -9.038e-0 |  2.477    |
|  3        | -8.577e-0 |  40.0     |
Period time: 0.2 minutes
Period: 4
|   iter    |  target   | look_back |
------------------------------------

|  1        | -6.319e-0 |  5.985    |
|  2        | -6.334e-0 |  38.66    |
|  3        | -8.415e-0 |  1.0      |
Period time: 0.52 minutes
Period: 9
|   iter    |  target   | look_back |
-------------------------------------
|  1        | -4.576e-0 |  19.14    |
|  2        | -4.586e-0 |  36.42    |
|  3        | -6.631e-0 |  1.0      |
Period time: 0.83 minutes
Period: 10
|   iter    |  target   | look_back |
-------------------------------------
|  1        | -4.916e-0 |  27.77    |
|  2        | -5.137e-0 |  7.009    |
|  3        | -4.899e-0 |  40.0     |
Period time: 1.51 minutes
Period: 11
|   iter    |  target   | look_back |
-------------------------------------
|  1        | -7.572e-0 |  7.948    |
|  2        | -7.584e-0 |  31.58    |
|  3        | -7.619e-0 |  39.99    |
Period time: 0.68 minutes
Period: 12
|   iter    |  target   | look_back |
-------------------------------------
|  1        | -5.833e-0 |  25.11    |
|  2        | -5.88e-06 |  4.031    |
|  3        | -5.

KeyboardInterrupt: 